1. 채무자키, 메모내용 읽기, 참조파일(전체원데이터, 보증인, 법조치 파산건 새로 다운) 읽기
2. 미종결 계좌가 있는 채무자키만 남기기
3. 파산사건번호 있는 메모추출(채무자키, 메모내용), 관할법원, 사건번호열 추가, 채무자이름열, 새채무자키 + 이름 + 사건번호 중복제거
4. 모든 참조 파일에 채무자키 대신 새채무자키
5. 법조치 기등록건 제외(새채무자키+사건번호--관할법원은 축어 많으니 제외)
6. 채무자키를 통해 채무자들 이름 병합 <------채무자, 보증인조회새창
7. 해당 메모에서 '차주','보증인','채무자이름'을 추가로 확인하여 대상자구분작성, 대표채무자키/보증인키 병합

### import 및 파일읽기

In [2]:
import os, re
import pandas as pd
import numpy as np
from os.path import join
import warnings
warnings.simplefilter("ignore")
from datetime import datetime
import functions

memo_dtype = {'채무자키':str, '메모키':str, "구분일":str}
momo_cols = ["메모키","채무자키","메모내용","등록일자"]
파산_cols = ["채무자키","관할법원","사건번호","법조치구분","법조치세부","사건구분","종국결과","비고","취하상태","접수일","종료일","종국결과일","대상자구분","대상자명"]

In [18]:
# # 메모 ###################################################
# 메모_파산_ori = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\메모조회새창_20240118_1802_파산.xlsx", dtype=memo_dtype)[momo_cols]
# 메모_하단_ori = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\메모조회새창_20240118_1803_하단.xlsx", dtype=memo_dtype)[momo_cols]
# 메모_하면_ori = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\메모조회새창_20240118_1803_하단.xlsx", dtype=memo_dtype)[momo_cols]

# 합치기, 중복제거, 등록일자 전처리, to pkl
# 메모_ori = pd.concat([메모_파산_ori, 메모_하단_ori, 메모_하면_ori], axis=0)
# 메모_ori["등록일자"] = 메모_ori.등록일자.str.extract(r'(\d{4,4}-\d\d-\d\d)')
# 메모_ori = 메모_ori.drop_duplicates(subset=["메모키"], keep='first', inplace=False, ignore_index=True)
# 메모_ori.to_pickle(r"C:\Users\DATA\Desktop\파산모음\메모.pkl")

# 파산 엑셀 to pkl
# event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
# 파산법조치 = pd.read_excel(r"C:\Users\DATA\Desktop\파산모음\파산모음_법조치조회새창_20240530_1804.xlsx", dtype=event_dtype)
# 파산법조치.to_pickle(r"C:\Users\DATA\Desktop\파산모음\파산모음.pkl")

In [ ]:
# 메모, 파산기등록건 법조치
메모 = pd.read_parquet(r"D:\3.자산\전산 dataset\대성\기간축적데이터\메모\parquet\대성메모전체_971213~250930.parquet")
# 참조파일 #################################################
base_dir = r"D:\3.자산\전산 dataset\대성\250930\parquet"
raw_data = pd.read_parquet(join(base_dir,"2.대성_전체_원데이터_250930_by새채무자키.parquet"))
# grt = pd.read_parquet(join(base_dir, "보증인새창_20251001_1238.parquet")) # rawdata에 보증인이름(unique) 있으므로 필요없음

### 메모 - 미종결건만 남기고 새채무자키 추가하기

In [5]:
# 미종결 계좌가 있는 채무자키만 남기기
미종결 = raw_data.query('새채무상태 != "종료"').copy()
# 미종결에서 채무자키 중복제거
미종결 = 미종결.drop_duplicates(subset=["채무자키"], ignore_index=True)

print("미종결메모----------")
# 메모 미종결건만 남기고 새채무자키 붙이기
미종결메모 = 메모.merge(미종결[["채무자키", "새채무자키"]], how='inner') # 기본값이 inner
print("전체메모건수", len(메모), "미종결메모건수",len(미종결메모))

미종결메모----------
전체메모건수 275722 미종결메모건수 54263


### 파산정보 담은 메모 추출

In [6]:
# 파산사건번호 컴파일
y_10 = str(datetime.today().year)[2] #23년이면 2
y_10_before = str(int(y_10)-1) #23년이면 2-1 = 1
y_1 = str(datetime.today().year)[3] # 23년이면 3
year_com = "([0-"+ y_10_before +"]\d|"+y_10+"[0-"+y_1+"])"

# 하면
p_하면4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하면)\s?(?P<sn>[0-9]+)") # 4자리 연도
p_하면2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하면)\s?(?P<sn>[0-9]+)") # 2자리 연도

# 하단
p_하단4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하단)\s?(?P<sn>[0-9]+)") # 4자리 연도
p_하단2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하단)\s?(?P<sn>[0-9]+)") # 2자리 연도

# 하
p_하4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하)\s?(?P<sn>[0-9]+)") # 4자리 연도
p_하2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하)\s?(?P<sn>[0-9]+)") # 2자리 연도

# 복합 : ex) 17하단 하면 4567 : 이런식으로 썼다는 것은 뒷자리가 같다는 거니까 es를 부르든, es2를 부르든 상관없다. 어차피 나중에 하면으로 통일해서 비교할거니
p_복합4=re.compile("((?<=\D)|^)(?P<y>19\d\d|20"+year_com+")\s?(?P<es>하[면단]).?(?P<es2>하?[면단])[^\d]?(?P<sn>[0-9]+)") # 4자리 연도
p_복합2=re.compile("((?<=\D)|^)(?P<y>[7-9][0-9]|"+year_com+")\s?(?P<es>하[면단]).?(?P<es2>하?[면단])[^\d]?(?P<sn>[0-9]+)") # 2자리 연도

def make_event_formular(case, re_object) :
    if case == 4: 
        event = re_object["y"] + re_object["es"] + re_object["sn"]
        # eSign = re_object["es"]
    elif case == 2:
        if int(re_object["y"])  <= int(y_10 + y_1):
            event = "20" + re_object["y"] + re_object["es"] + re_object["sn"]
        else :
            event = re_object["y"] + re_object["es"] + re_object["sn"] # 1900년대는 사건번호에 19안 붙음
        # eSign = re_object["es"]
    return event

comp = {"하면" : [p_하면4,p_하면2], "하단" : [p_하단4, p_하단2], "하" : [p_하4, p_하2], "복합" : [p_복합4, p_복합2]}

In [ ]:
#################################################
검색사건기호들 = ("하면","하단","하","복합")
#################################################

추출_df = pd.DataFrame(None)
for 검색사건기호 in 검색사건기호들 :
    추출 = []
    for i, row in 미종결메모.iterrows() : 
        
        event_set = set()
        
        # 4연도 사건번호 전부 추출
        res4 = list(comp[검색사건기호][0].finditer(row.메모내용)) 
        if res4 : 
            # 사건번호 규격화 하고, 고유값만 남기기
            
            for r in res4 : 
                form_event = make_event_formular(4, r)
                event_set.add(form_event)
        
        res2 = list(comp[검색사건기호][1].finditer(row.메모내용)) 
        
        # 2연도 사건번호 전부 추출
        if res2 : 
            # 사건번호 규격화 하고, 고유값만 남기기
            for r in res2 : 
                form_event = make_event_formular(2, r)
                event_set.add(form_event)
            
        # 규격화된 사건번호의 고유값 개수에 따라 행을 추가하여 df만들기
        for e in list(event_set) :
            new_row = row.copy()
            new_row["사건번호"] = e
            추출.append(new_row)
    
    추출 = pd.DataFrame(추출)  # 리스트가 Series나 dict이면 바로 DataFrame 생성 가능

    if len(추출) > 0 : # 추출이 none면 에러남.
        추출_df = pd.concat([추출_df, 추출], axis=0)

### 전처리 : 관할법원, 사건번호, 대상자, 누락여부

In [ ]:
###########################
company = "대성"
###########################
# 관할법원
p_court = re.compile(r"(([가-힣]+(법원|지원|지법))|(?:(?:\s|\b)?(서울|대전|대구|부산|광주|수원|\
        |의정부|파주|포천|동두천|가평|연천|철원|인천|김포|강화|용인|오산|광명|\
        |안성|양평|이천|춘천|홍천|양구|삼척|동해|정선|평창|태백|횡성|인제|화천|\
        |고성|양양|금산|세종|보령|서천|예산|아산|태안|당진|(?<![도통문])부여|청양|진천|보은|\
        |단양|음성|옥천|괴산|경산|칠곡|청도|영천|성주|고령|영주|봉화|구미|문경|\
        |예천|청송|군위|울진|영양|양산|울산|창원|마산|김해|함안|의령|사천|남해|\
        |하동|거제|창녕|합천|함양|산청|담양|함평|강진|구례|영광|나주|장성|화순|\
        |곡성|광양|고흥|여수|보성|무안|영암|완도|진도|전주|군산|정읍|남원|진안|\
        |김제|무주|임실|익산|부안|고창|장수|순창|제주|서귀포|청주)[가-힣]*))")

    # 지법 > 지방법원
추출_df["관할법원"] = 추출_df.메모내용.str.extract(p_court)[0].str.strip()
추출_df["관할법원"] = 추출_df["관할법원"].replace("지법", "지방법원", regex=True)

cond_suwon = 추출_df["관할법원"].str.contains("수원", na=False)
cond_busan = 추출_df["관할법원"].str.contains("부산", na=False)
cond_seoul = 추출_df["관할법원"].str.contains("서울", na=False)

    # 회생법원
추출_df["관할법원"] = np.select(
    [cond_suwon, cond_busan, cond_seoul],
    ["수원회생법원", "부산회생법원", "서울회생법원"],
    default=추출_df["관할법원"]
)

# 사건번호 : 하면변환, 중복사건번호 삭제
추출_df["사건번호_하면"] = 추출_df["사건번호"].replace({"하단":"하면"}, regex=True) # 부분을 치환할 때는 regex 필수
        # 첫행만 남기기 : 채무자포함여부, 보증인포함여부 정렬했으므로 위에꺼 살리면 됨
print("채무자키-사건번호 중복 삭제전",len(추출_df))
추출_df = 추출_df.drop_duplicates(subset=["채무자키","사건번호_하면"], keep='first') 
print("채무자키-사건번호 중복 삭제후",len(추출_df))

# 대상자, 누락여부
    # 정확한 신청인명은 누락여부 확인하면서 한번더, 대상자는 최신화 결과를 통해서 재확인
    # 보증인성명인 붙이기 : 중복제거 필수
print('보증인성명인 병합전',len(추출_df))
추출_df_merged = 추출_df.merge(raw_data[["채무자키", "보증인성명인"]].drop_duplicates(subset=["채무자키"]), how='left', on='채무자키')
print('보증인성명인 병합후',len(추출_df_merged))

    # 차주 사건여부
추출_df_merged["채무자명포함여부"] = 추출_df_merged.apply(
    lambda row : row["채무자명"] in row["메모내용"],
    axis=1)

    # 보증인 사건여부
def 보증인포함여부(row):
    memo = row["메모내용"]
    names = [n.strip() for n in str(row["보증인성명인"]).split(",") if n.strip()] + ["보증인"] # 이름 리스트화
    포함된이름 = [name for name in names if name in memo]
    return ",".join(포함된이름) if 포함된이름 else "" 

추출_df_merged["보증인포함여부"] = 추출_df_merged.apply(보증인포함여부, axis=1)

    # 누락여부 '포함', '누락' search
추출_df_merged["당사포함여부"] = np.select(
    [추출_df_merged["메모내용"].str.contains(r"(?<!미)포함|수령|이의신청기간|"+company, na=False), 추출_df_merged["메모내용"].str.contains(r"누락|미포함|제외", na=False)],
    ["포함", "누락"],
    default=""
)

삭제전 432
삭제후 279
279
279


### 법조치 미등록건만 남기기

In [ ]:
# 파산등록건 읽기
파산등록건 = pd.read_parquet(r"D:\3.자산\전산 dataset\대성\250930\통합dc_for_v2\법조치조회새창_20251028_1642.parquet")[파산_cols]

# 파산등록건에 새채무자키 붙이기
print("전체", len(파산등록건))
파산등록건 = 파산등록건.merge(미종결[["채무자키","새채무자키"]], how='inner')
print("미종결건", len(파산등록건))

# 비교편의 위해 하면으로 변환(실제 하단을 하면으로 바꿔야하는 경우는 자산정리 dc파일 통해)
파산등록건["법조치사건번호_하면"] = 파산등록건["사건번호"].replace({"하단":"하면"}, regex=True)

# 파산등록건 중 새채무자키와 사건번호_하면 중복제거
파산등록건.drop_duplicates(subset=["새채무자키", "법조치사건번호_하면"], inplace=True, ignore_index=True)
print("새채무자키,사건번호 중복건 제거 후", len(파산등록건))

# 파산등록건에 등록되지 않은 사건번호만 남기기
추출_파산등록건 = 추출_df_merged.merge(파산등록건[["새채무자키","법조치사건번호_하면","법조치구분"]], left_on=["새채무자키","사건번호_하면"], right_on=["새채무자키","법조치사건번호_하면"], indicator=True, how='outer')
print(추출_파산등록건._merge.value_counts(), 'left_only:파산미등록건')
미등록건 = 추출_파산등록건.query('_merge=="left_only"')
미등록건.drop(columns=["_merge"], inplace=True)

전체 185
미종결건 162
새채무자키,사건번호 중복건 제거 후 162


In [ ]:
# 엑셀저장
    # 정렬
미등록건.sort_values(by=['채무자키','관할법원','사건번호','채무자명포함여부','보증인포함여부'], ascending=[True,True,False, False, True], inplace=True) # 하면이 위에오도록 사건번호는 f, t/f값은 f가 먼저 나옴(0이니까)
    # 저장 칼럼
cols = ["채무자키","등록일자","메모내용","메모구분","채무자명","새채무자키","관할법원","사건번호","사건번호_하면","채무자명포함여부","보증인포함여부","보증인성명인","당사포함여부"]
    
    # 저장
fullpath_write = r"D:\3.자산\전산 dataset\대성\250930\통합dc_for_v2\미등록파산건_251028.xlsx"
functions.save_df_to_excel_underline(미등록건[cols], fullpath_write)